In [6]:
from __init__ import *
%pdb off
%matplotlib inline 
print (os.getenv('dataroot'))
print (os.getenv('CONDA_PROMPT_MODIFIER'), datetime.now())

Automatic pdb calling has been turned OFF
/Users/buzzanga/data
(MintPy)  2022-03-15 11:59:37.029354


In [7]:
## keep all exps in dct_exps and then select with below
dct_exps  = {
    'HR_Exps': dict(root='HR_Exps', nx=11, ny=5, naz=7, nrg=19, gap_fill='_filled'),
    'HR_Exps1': dict(root='HR_Exps', nx=11, ny=5, naz=2, nrg=5, gap_fill='_filled'),

    'HR0': dict(root='HR', nx=5, ny=5, naz=2, nrg=5, gap_fill='_filled'),
    'HR1': dict(root='HR', nx=11, ny=5, naz=2, nrg=5, gap_fill='_filled'),
    'HR2': dict(root='HR', nx=33, ny=15, naz=2, nrg=5, gap_fill='_filled'),
    'HR2a': dict(root='HR', nx=33, ny=15, naz=3, nrg=5, gap_fill='_filled'),
    'HR16': dict(root='HR_2016', nx=11, ny=5, naz=2, nrg=5, gap_fill='_filled')    
    }

In [8]:
# ======================= #
dct      = dct_exps['HR2']
ts_ext   = '_demErr'
# ======================= #

nx, ny   = dct['nx'], dct['ny']  # half window size
naz, nrg = dct['naz'], dct['nrg'] # looks
gap_fill = dct['gap_fill']
ml, nxy  = f'{naz}alks_{nrg}rlks', f'_{nx}_{ny}'


path_vlm   = op.join(os.getenv('dataroot'), 'VLM')
wd         = op.join(path_vlm, *f'Sentinel1 {dct["root"]}'.split())
path_wraps = op.join(wd, f'PS_DS{nxy}')


path_gps = op.join(path_vlm, 'GNSS', 'UNR', f'Midas_GPS-HR_IGS14_crop.csv')

path_mp  = op.join(wd, f'MintPy_{ml}{nxy}')

path_vup        = op.join(path_mp, 'Vup')


path_vup_geo    = op.join(path_vup, 'geo_Vup.h5')
path_mask_vup   = op.join(path_vup, 'geo_Vup_mask.h5')  # combined mask
path_mask_tcgeo = op.join(path_mp, 'geo_maskTempCoh.h5')
path_mask_tcgeo_nc = path_mask_tcgeo.replace('.h5', '.nc')

path_ts       = op.join(path_mp, f'timeseries{ts_ext}.h5')
path_ts_geo   = op.join(path_mp, f'geo_timeseries{ts_ext}.h5')
path_ts_swift = op.join(path_mp, f'geo_timeseries{ts_ext}_swift.h5')


path_rate_grd = op.join(path_vup, 'geo_rate.grd')
path_rate_nc  = op.join(path_vup, 'geo_rate.nc')
path_rate_msk = op.join(path_vup, 'geo_rate_masked.nc') # applied mask to vel
path_rate_kmz = op.join(path_vup, 'geo_rate') 

path_std_grd = op.join(path_vup, 'geo_std.grd')
path_std_nc  = op.join(path_vup, 'geo_std.nc')
path_std_msk = op.join(path_vup, 'geo_std_masked.nc') # applied mask to std
path_std_kmz = op.join(path_vup, 'geo_std') 

path_mask_vup_nc  = op.join(path_vup, 'geo_Vup_mask.nc')  # combined mask

path_transects  = op.join(path_vup, 'Transects')

path_wmask = op.join(path_mp, 'geo_waterMask.nc')  # for GMT background

## move
os.makedirs(path_mp, exist_ok=True)
os.chdir(path_mp)
print ('Path_Mint (PWD):', path_mp)

Path_Mint (PWD): /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15


## Prep Data

In [9]:
# reference timeseries to extensometer location
ref_lalo = [36.89375, -76.4211]
cmd = f'reference_point.py {path_ts_geo} -o {path_ts_swift} -l {ref_lalo[0]}  -L {ref_lalo[1]}'
print (cmd)
!{cmd}

reference_point.py /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_timeseries_demErr.h5 -o /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_timeseries_demErr_swift.h5 -l 36.89375  -L -76.4211
--------------------------------------------------
input reference point in lat/lon: (36.89375, -76.4211)
input reference point in y/x: (2258, 797)
--------------------------------------------------
calculating the temporal average of timeseries file: /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_timeseries_demErr.h5
reading timeseries data from file: /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_timeseries_demErr.h5 ...
/Users/buzzanga/Software_InSAR/MintPy_git/mintpy/objects/stack.py:440: RuntimeWarning: Mean of empty slice
  dmean = np.nanmean(data, axis=0)
writing the referenced data into file: /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_timeseries_demErr_swift.h5
create HDF5 file: /Users/buzz

In [23]:
# make sure the waterMask netcdf exists for cropping
cmd = f'save_gdal.py {path_wmask.replace(".nc", ".h5")} -o {path_wmask} --of netCDF'

print (cmd)
!{cmd}

save_gdal.py /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_waterMask.h5 -o /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_waterMask.nc --of netCDF
read metadata from file: /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_waterMask.h5
read data     from file: /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_waterMask.h5
initiate GDAL driver: Network Common Data Format
create raster band
raster row / column number: 3495, 2479
raster transform info: (-76.65497753137032, 0.00029319750626338167, 0, 37.30743028377248, 0, -0.0001831383457471231)
write data to raster band
set projection as: EPSG 4326
finished writing to /Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_waterMask.nc


In [19]:
from VLM.EC import Swift_Analysis
Swift_Analysis.SwiftInject(path_ts_swift)
path_ts_swift

'/Users/buzzanga/data/VLM/Sentinel1/HR/MintPy_2alks_5rlks_33_15/geo_timeseries_demErr_swift.h5'